# ISLP - Chapter 13 - Exercise 7
### Author: pzuehlke

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

In [2]:
carseats = pd.read_csv("Carseats.csv")
carseats.dropna(inplace=True)
carseats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 34.5+ KB


In [3]:
carseats.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


__7 (a):__ Let's filter out the quantitative predictors:

In [4]:
quant_vars = carseats.select_dtypes(include=["number"]).columns.tolist()
quant_vars.remove("Sales")  # Remove the response variable
print(len(quant_vars))
print(quant_vars)

7
['CompPrice', 'Income', 'Advertising', 'Population', 'Price', 'Age', 'Education']


We not fit a linear model $ \text{Sales} = \alpha + \beta X + \epsilon $ for
each of the $ 7 $ quantitative predictors $ X $ listed above and compute the
corresponding $ p $-value of the null hypothesis $ H_0 : \beta = 0 $.

In [5]:
p_values = []
variable_names = []
y = carseats["Sales"]

# Fit linear models for each quantitative variable:
for var in quant_vars:
    X = sm.add_constant(carseats[var])
    model = sm.OLS(y, X).fit()
    # Store the p-value for the variable coefficient (not intercept):
    p_values.append(model.pvalues.iloc[1])
    variable_names.append(var)
    
p_values = np.array(p_values)

# Create a DataFrame to store results:
results = pd.DataFrame({
    "Variable": variable_names,
    "p_value": p_values,
    "Reject": p_values < 0.05
}).sort_values(by="p_value")

print("\nSummary of p-values for all variables:")
results


Summary of p-values for all variables:


,Variable,p_value,Reject
4,Price,7.618187e-21,True
2,Advertising,4.377677e-08,True
5,Age,2.788950e-06,True
1,Income,2.309670e-03,True
0,CompPrice,2.009398e-01,False
6,Education,2.999442e-01,False
3,Population,3.139816e-01,False


__7 (b):__ From the preceding table of $ p $-values, using a threshold $ \alpha = 0.05 $ for rejection of the null hypotheses, we would reject the null hypothesis that the linear coefficient is zero for the variables: `Price`, `Advertising`, `Age` and `Income`, i.e., for $ 4 $ out of $ 7 $ of the quantitative variables.

__7 (c):__ We use Bonferroni's method to control the FWER at the same level of
$ 0.05 $ for rejection, storing the adjusted $ p $-values and the FWER veredict
for the rejections in the same dataframe as in item (a). 

In [6]:
p_values.sort()
reject_fwer, adj_p_values = multipletests(p_values, method="bonferroni", alpha=0.05)[:2]
results["Bonferroni_p"] = adj_p_values
results["Reject_FWER"] = reject_fwer

results

,Variable,p_value,Reject,Bonferroni_p,Reject_FWER
4,Price,7.618187e-21,True,5.332731e-20,True
2,Advertising,4.377677e-08,True,3.064374e-07,True
5,Age,2.788950e-06,True,1.952265e-05,True
1,Income,2.309670e-03,True,1.616769e-02,True
0,CompPrice,2.009398e-01,False,1.000000e+00,False
6,Education,2.999442e-01,False,1.000000e+00,False
3,Population,3.139816e-01,False,1.000000e+00,False


The variables that are still significant at the FWER level of $ 0.05 $ using
Bonferroni are the same as before:

In [7]:
print(results[results["Reject_FWER"]]["Variable"].tolist())

['Price', 'Advertising', 'Age', 'Income']


We will also manually compute the adjusted $ p $-values ($ = 7 \times $ the
original $ p $-values) to check if they coincide with those calculated by statsmodels:

In [8]:
num_tests = len(quant_vars)
manual_bonferroni = np.minimum(p_values * num_tests, 1.0)
comparison = pd.DataFrame({
    "Variable": variable_names,
    "Original_p": p_values,
    "Manually adj. p_values": manual_bonferroni,
    "sm adj. p_values": adj_p_values
}).sort_values(by="Original_p")
comparison

,Variable,Original_p,Manually adj. p_values,sm adj. p_values
0,CompPrice,7.618187e-21,5.332731e-20,5.332731e-20
1,Income,4.377677e-08,3.064374e-07,3.064374e-07
2,Advertising,2.788950e-06,1.952265e-05,1.952265e-05
3,Population,2.309670e-03,1.616769e-02,1.616769e-02
4,Price,2.009398e-01,1.000000e+00,1.000000e+00
5,Age,2.999442e-01,1.000000e+00,1.000000e+00
6,Education,3.139816e-01,1.000000e+00,1.000000e+00


__7 (d)__: Finally, we control the FDR at the threshold $ q = 20\% $
using the Benjamini-Hochberg method. Basically we repeat the code
in item (c), but passing the argument `method="fdr_bh"` in the call to
`multipletests`:

In [9]:
reject_fdr, q_values = multipletests(results["p_value"],
                                     method="fdr_bh", alpha=0.2)[:2]
results["BH_q"] = q_values
results["Reject_FDR"] = reject_fdr
print(results[results["Reject_FDR"]]["Variable"].tolist())

['Price', 'Advertising', 'Age', 'Income']


Using this method we again reject the same $ 4 $ null hypotheses, so for this
problem there is no difference at all among the three methods that we
considered, even though in general the outcomes are different. We finish by
displaying the full results:

In [10]:
results

,Variable,p_value,Reject,Bonferroni_p,Reject_FWER,BH_q,Reject_FDR
4,Price,7.618187e-21,True,5.332731e-20,True,5.332731e-20,True
2,Advertising,4.377677e-08,True,3.064374e-07,True,1.532187e-07,True
5,Age,2.788950e-06,True,1.952265e-05,True,6.507550e-06,True
1,Income,2.309670e-03,True,1.616769e-02,True,4.041923e-03,True
0,CompPrice,2.009398e-01,False,1.000000e+00,False,2.813158e-01,False
6,Education,2.999442e-01,False,1.000000e+00,False,3.139816e-01,False
3,Population,3.139816e-01,False,1.000000e+00,False,3.139816e-01,False
